In [1]:

import torch
import os

import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

file_id=6448

base_path="/home/eloi/projects/project_mfm_eloi/audio_examples/TM_val/"
file_dry=os.path.join(base_path, "dry",str(file_id),"vocals.wav")
file_wet=os.path.join(base_path, "wet",str(file_id),"vocals.wav")

file_dry_fxnorm=os.path.join(base_path,"dry", str(file_id), "vocals_normalized.wav")
file_wet_fxnorm=os.path.join(base_path, "wet",str(file_id), "vocals_normalized.wav")

file_dry_fxnorm_dr=os.path.join(base_path, "dry", str(file_id), "vocals_normalized_dr.wav")
file_wet_fxnorm_dr=os.path.join(base_path, "wet",str(file_id),"vocals_normalized_dr.wav")

import soundfile as sf

dry, sr = sf.read(file_dry)
wet, sr = sf.read(file_wet)

file_dry_fxnorm, sr=sf.read(file_dry_fxnorm_dr)
file_wet_fxnorm, sr=sf.read(file_wet_fxnorm_dr)

dry=torch.from_numpy(dry.T).float().unsqueeze(0)
wet=torch.from_numpy(wet.T).float().unsqueeze(0)

dry_fxnorm=torch.from_numpy(file_dry_fxnorm.T).float().unsqueeze(0)

dry=dry.mean(dim=1, keepdim=True)

dry_fxnorm=torch.from_numpy(file_dry_fxnorm.T).float().unsqueeze(0)




import pyloudnorm as pyln
meter = pyln.Meter(sr)
normaliser = lambda x: pyln.normalize.loudness(
    x, meter.integrated_loudness(x), -18.0
)

print("dry", dry.shape, dry.min(), dry.max())

dry = torch.from_numpy(normaliser(dry.numpy().T).T).float().to(device)
#wet = torch.from_numpy(normaliser(wet.numpy().T).T).float().to(device)
dry_fxnorm = torch.from_numpy(normaliser(dry_fxnorm.numpy().T).T).float().to(device)


print("dry", dry.shape,dry.min(), dry.max())

start_t=15*sr
segment_length = 525312
dry_segment = dry[...,start_t:start_t + segment_length]
wet_segment = wet[...,start_t:start_t + segment_length]

dry torch.Size([1, 1, 11065479]) tensor(-0.1188) tensor(0.1767)


/home/eloi/myenv/lib/python3.11/site-packages/pyloudnorm/normalize.py:62: UserWarning: Possible clipped samples in output.
  warnings.warn("Possible clipped samples in output.")


dry torch.Size([1, 1, 11065479]) tensor(-0.7274, device='cuda:0') tensor(1.0821, device='cuda:0')


In [2]:
dry.shape

torch.Size([1, 1, 11065479])

In [2]:

from fx_model.dasp_diffvox import DASPDiffVox

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=DASPDiffVox(sample_rate=44100).to(device)

xout=model(dry_segment.to(device))

print("xout", xout.shape, xout.min(), xout.max())


/home/eloi/myenv/lib/python3.11/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


xout torch.Size([1, 1, 525312]) tensor(-0.3862, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.3537, device='cuda:0', grad_fn=<MaxBackward1>)
